<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [90]:
import pandas as pd
import psycopg2

In [91]:
import warnings
warnings.filterwarnings('ignore')

In [92]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python

In [94]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [98]:
# текст запроса
query_3_1 = f'''SELECT COUNT(*) AS total_vacancies FROM vacancies
'''

In [99]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

,total_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [100]:
# текст запроса
query_3_2 = f'''SELECT COUNT(*) AS total_employers FROM employers
'''

In [101]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

,total_employers
0,23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [102]:
# текст запроса
query_3_3 = f'''SELECT COUNT(*) AS total_areas FROM areas
'''

In [103]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

,total_areas
0,1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [104]:
# текст запроса
query_3_4 = '''SELECT COUNT(*) AS total_industries FROM industries'''

In [105]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

,total_industries
0,294


***

In [106]:
# выводы по предварительному анализу данных

В базе данных содержатся записи по 49 тыс выкансий и 23,5 тыс работодателей, соответственно, некоторые работодатели публикуют по несколько вакансий.

Если речь о вакансиях только в сфере Data Science, то это достаточно большое число.

География вакансий достаточно обширна (1362 уникальных региона)

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [107]:
# текст запроса
query_4_1 = '''
SELECT
    a.name AS area,
    COUNT(v.id) AS cnt
FROM vacancies v
JOIN areas a ON v.area_id = a.id  -- Связь с таблицей регионов
GROUP BY a.name
ORDER BY cnt DESC;
'''

In [108]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [109]:
# текст запроса
query_4_2 = '''
SELECT
    COUNT(*) AS cnt
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;
'''

In [110]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [111]:
# текст запроса
query_4_3 = '''
SELECT
    ROUND(AVG(salary_from))::integer AS avg_salary_from,
    ROUND(AVG(salary_to))::integer AS avg_salary_to
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL;  -- Исключаем вакансии без зарплат
'''

In [112]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

,avg_salary_from,avg_salary_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [113]:
# текст запроса
query_4_4 = '''
SELECT
    schedule,
    employment,
    COUNT(*) AS cnt
FROM vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC;
'''

In [114]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [115]:
# текст запроса
query_4_5 = '''
SELECT
    experience,
    COUNT(*) AS cnt
FROM vacancies
GROUP BY experience
ORDER BY cnt ASC;  -- Сначала редкие варианты опыта
'''

In [116]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [117]:
# выводы по детальному анализу вакансий

География вакансий достаточно обширна (1362 уникальных региона), при этом около 10.8% сконцентрированы в Москве

Вообще, вакансии сконцентрированы в крупных городах (лидеры: Москва (5 тыс), Санкт-Петербург (~3 тыс), Минск (2 тыс))

60% всех регионов (769 из 1,362) имеют менее 10 вакансий. Это указывает ограниченные возможности в регионах

Только 49% вакансий (24 тыс из 49 тыс) указывают зарплату (хотя бы одну границу). При этом там, где границы указаны, существует довольно большой разброс (от 71 тыс до 110 тыс)

Среди форматов занятости доминирует "полный день + полная занятость" (72% или \~35 тыс вакансий). Удаленная работа на втором месте (\~8 тыс)

Рынку работодателей требуются middle специалисты: больше всего вакансий для специалистов с относительно небольшим опытом: от 1 до 3 (53%, 26 тыс) и от 3 до 6 (29%, 14.5 тыс). Без опыта (15%, 7 тыс). Меньше всего требуются senior специалисты (2.7%, \~3 тыс)

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [118]:
# текст запроса
query_5_1 = '''
SELECT
    e.name AS employer_name,
    COUNT(v.id) AS vacancies_count
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
GROUP BY e.id, e.name
ORDER BY vacancies_count DESC
LIMIT 5;
'''

In [119]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

,employer_name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [120]:
# текст запроса
query_5_2 = '''
SELECT
    a.name AS area_name,
    COUNT(e.id) AS employers_count,
    0 AS vacancies_count
FROM areas a
JOIN employers e ON e.area = a.id
WHERE NOT EXISTS (
    SELECT 1 FROM vacancies v
    WHERE v.area_id = a.id
)
GROUP BY a.id, a.name
ORDER BY employers_count DESC
LIMIT 1;
'''

In [121]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

,area_name,employers_count,vacancies_count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [122]:
# текст запроса
query_5_3 = '''
SELECT
    e.name AS employer_name,
    COUNT(DISTINCT v.area_id) AS regions_count
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
GROUP BY e.id, e.name
ORDER BY regions_count DESC;
'''

In [123]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

,employer_name,regions_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [124]:
# текст запроса
query_5_4 = '''
SELECT
    COUNT(*) AS employers_without_industry
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.employer_id IS NULL;
'''

In [125]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

,employers_without_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [126]:
# текст запроса
query_5_5 = '''
SELECT
    e.name AS employer_name
FROM employers e
JOIN (
    SELECT
        employer_id
    FROM employers_industries
    GROUP BY employer_id
    HAVING COUNT(industry_id) = 4
) ei ON e.id = ei.employer_id
ORDER BY e.name ASC
OFFSET 2 LIMIT 1;
'''

In [127]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

,employer_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [128]:
# текст запроса
query_5_6 = '''
SELECT
    COUNT(DISTINCT ei.employer_id) AS employers_count
FROM employers_industries ei
JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения';
'''

In [129]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

,employers_count
0,3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [130]:
import requests
import json
from bs4 import BeautifulSoup

In [131]:
# код для получения списка городов-милионников
# Запрос к API Википедии
api_url = "https://ru.wikipedia.org/w/api.php"
params = {
    'action': 'parse',
    'page': 'Города-миллионеры_России',
    'prop': 'text',
    'format': 'json',
    'formatversion': 2
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(api_url, params=params, headers=headers)
data = response.json()

# Извлекаем HTML из ответа API
html_content = data['parse']['text']

# Парсим HTML с помощью BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Находим таблицу с городами
table = soup.find('table', class_='standard')

# Извлекаем названия городов из таблицы
cities = []
if table:
    for row in table.find_all('tr')[1:]:  # Пропускаем заголовок таблицы
        cells = row.find_all('td')
        if len(cells) >= 2:
            city_link = cells[1].find('a')
            if city_link:
                cities.append(city_link.text)

print(cities)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [132]:
# Преобразуем список в строку для SQL-запроса
cities_str = "', '".join(cities)
cities_str

"Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград"

In [133]:
# текст запроса
query_5_7 = f'''
SELECT
    COALESCE(a.name, 'Total') AS city,
    COUNT(v.id) AS vacancies_count
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
JOIN areas a ON v.area_id = a.id
WHERE
    e.name ILIKE 'Яндекс' AND
    a.name IN ('{cities_str}')
GROUP BY GROUPING SETS ((a.name), ())
ORDER BY GROUPING(a.name), vacancies_count ASC;
'''

In [134]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

,city,vacancies_count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

In [135]:
# выводы по анализу работодателей

Компания Яндекс лидирует: 1933 - почти в 4 раза больше, чем у второго по величине работодателя (Ростелеком, 491)

Топ работодателей по секторам:
- IT (Яндекс), данному сектору принадлежат 15% (\~3.5 тыс) вакансий
- телекоммуникации (Ростелеком)
- банки/финтех (Сбер, Тиньков)
- энергетика (Газпром Нефть)

В "России" нет ни одной вакансии - вероятно, обобщающая категория без привязки к конкретному региону

Москва и СПб вместе составляют почти 20% всех вакансий Яндекса в городах-миллионниках

36% работодателей (\~8.5 тыс из 23.5)  не указали сферу деятельности.

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [136]:
# текст запроса
query_6_1 = '''
SELECT COUNT(*) AS data_vacancies_count
FROM vacancies
WHERE name ILIKE '%data%' OR name ILIKE '%данн%';
'''

In [137]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

,data_vacancies_count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [138]:
# текст запроса
query_6_2 = '''
SELECT COUNT(*) AS junior_ds_vacancies_count
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' OR
    name ILIKE '%data science%' OR
    name ILIKE '%исследователь данных%' OR
    (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
)AND (
    name ILIKE '%junior%' OR
    experience = 'Нет опыта' OR
    employment = 'Стажировка'
);
'''

In [139]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

,junior_ds_vacancies_count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [140]:
# текст запроса
query_6_3 = '''
SELECT COUNT(*) AS ds_sql_vacancies_count
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' OR
    name ILIKE '%data science%' OR
    name ILIKE '%исследователь данных%' OR
    (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
) AND (
    key_skills ILIKE '%SQL%' OR
    key_skills ILIKE '%Postgres%'
);
'''

In [141]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

,ds_sql_vacancies_count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [142]:
# текст запроса
query_6_4 = '''
SELECT COUNT(*) AS ds_python_vacancies_count
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' OR
    name ILIKE '%data science%' OR
    name ILIKE '%исследователь данных%' OR
    (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
) AND key_skills ILIKE '%Python%';
'''

In [143]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

,ds_python_vacancies_count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [144]:
# текст запроса
query_6_5 = '''
SELECT ROUND(AVG(ARRAY_LENGTH(regexp_split_to_array(key_skills, ','), 1)), 2) AS avg_skills_count
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' OR
    name ILIKE '%data science%' OR
    name ILIKE '%исследователь данных%' OR
    (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
) AND key_skills IS NOT NULL;
'''

In [145]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

,avg_skills_count
0,1.09


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [146]:
# текст запроса
query_6_6 = '''
SELECT
    ROUND(AVG(
        (COALESCE(salary_from::numeric, 0) + COALESCE(salary_to::numeric, 0)) /
        NULLIF((CASE WHEN salary_from IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN salary_to IS NOT NULL THEN 1 ELSE 0 END), 0)
    ))::integer AS avg_salary
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' OR
    name ILIKE '%data science%' OR
    name ILIKE '%исследователь данных%' OR
    (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%') OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
) AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
AND experience = 'От 3 до 6 лет';
'''

In [147]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
df

,avg_salary
0,256454


***

In [148]:
# выводы по предметному анализу

3.6% (1771) вакансий связаны с данными, 20% из них требуют Python (скорее, 20% только указали Python, но сам Python требуется везде), в \~13% требуется SQL

Среднее число навыков, указанное в вакансиях - 1.09 (возможно ошибка, т.к. на платформе отмечено как неверный ответ)

Средняя з/п 256 тыс. руб., что в 3.7 раза больше нижней границы (71 тыс)



## Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

In [149]:
# текст запроса
query_additional_1 = '''
SELECT
    a.name AS region,
    ROUND(AVG(
        (COALESCE(v.salary_from::numeric, 0) + COALESCE(v.salary_to::numeric, 0)) /
        NULLIF((CASE WHEN v.salary_from IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN v.salary_to IS NOT NULL THEN 1 ELSE 0 END), 0)
    ))::integer AS avg_salary
FROM vacancies v
JOIN areas a ON v.area_id = a.id
WHERE (
    v.name ILIKE '%data scientist%' OR
    v.name ILIKE '%data science%' OR
    v.name ILIKE '%исследователь данных%' OR
    v.name ILIKE '%machine learning%' OR
    v.name ILIKE '%машинн%обучен%'
) AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
GROUP BY a.name
HAVING COUNT(*) >= 5  -- Только регионы с достаточным количеством вакансий
ORDER BY avg_salary DESC
LIMIT 10;
'''

In [151]:
# результат запроса
df = pd.read_sql_query(query_additional_1, connection)
df

,region,avg_salary
0,Москва,208583
1,Санкт-Петербург,173058


In [152]:
# текст запроса
query_additional_2 = '''
SELECT
    experience,
    COUNT(*) AS vacancies_count,
    ROUND(AVG(
        (COALESCE(salary_from::numeric, 0) + COALESCE(salary_to::numeric, 0)) /
        NULLIF((CASE WHEN salary_from IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN salary_to IS NOT NULL THEN 1 ELSE 0 END), 0)
    ))::integer AS avg_salary,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY
        (COALESCE(salary_from::numeric, 0) + COALESCE(salary_to::numeric, 0)) /
        NULLIF((CASE WHEN salary_from IS NOT NULL THEN 1 ELSE 0 END +
                CASE WHEN salary_to IS NOT NULL THEN 1 ELSE 0 END), 0)
    ))::integer AS median_salary
FROM vacancies
WHERE (
    name ILIKE '%data scientist%' OR
    name ILIKE '%data science%' OR
    name ILIKE '%исследователь данных%' OR
    name ILIKE '%machine learning%' OR
    name ILIKE '%машинн%обучен%'
) AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
ORDER BY
    CASE experience
        WHEN 'Нет опыта' THEN 1
        WHEN 'От 1 года до 3 лет' THEN 2
        WHEN 'От 3 до 6 лет' THEN 3
        WHEN 'Более 6 лет' THEN 4
    END;
'''

In [153]:
# результат запроса
df = pd.read_sql_query(query_additional_2, connection)
df

,experience,vacancies_count,avg_salary,median_salary
0,Нет опыта,7,74643,72500
1,От 1 года до 3 лет,23,146689,143500
2,От 3 до 6 лет,23,253129,233794


In [154]:
# текст запроса
query_additional_3 = '''
SELECT
    e.name AS employer_name,
    COUNT(DISTINCT a.name) AS regions_count,
    STRING_AGG(DISTINCT a.name, ', ') FILTER (WHERE a.name NOT IN ('Москва', 'Санкт-Петербург')) AS other_regions
FROM vacancies v
JOIN employers e ON v.employer_id = e.id
JOIN areas a ON v.area_id = a.id
WHERE a.name NOT IN ('Москва', 'Санкт-Петербург')
GROUP BY e.name
HAVING COUNT(DISTINCT a.name) >= 5
ORDER BY regions_count DESC
LIMIT 10;
'''

In [155]:
# результат запроса
df = pd.read_sql_query(query_additional_3, connection)
df

,employer_name,regions_count,other_regions
0,Яндекс,179,"Абакан, Актау, Актобе, Альметьевск, Ангарск, А..."
1,Ростелеком,150,"Абакан, Аликово, Ангарск, Архангельск, Астраха..."
2,Спецремонт,115,"Абакан, Альметьевск, Ангарск, Арзамас, Армавир..."
3,Поляков Денис Иванович,87,"Ангарск, Артем, Архангельск, Астрахань, Балако..."
4,ООО ЕФИН,71,"Алматы, Арзамас, Архангельск, Астрахань, Балаш..."
5,Совкомбанк,62,"Агинское (Забайкальский АО), Адыге-Хабль, Алда..."
6,МТС,53,"Абакан, Анапа, Архангельск, Барнаул, Белгород,..."
7,"ЭФКО, Управляющая компания",48,"Алексеевка, Архангельск, Барнаул, Белгород, Бр..."
8,КРОН,47,"Альметьевск, Архангельск, Астрахань, Барнаул, ..."
9,MCORE,46,"Арсеньев, Баргузин, Барнаул, Беслан, Биробиджа..."


In [163]:
query_additional_4 = '''
SELECT
    employment,
    experience,
    COUNT(*) AS vacancies_count,
    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2))::integer AS avg_salary
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
GROUP BY employment, experience
HAVING COUNT(*) >= 5
ORDER BY
    CASE employment
        WHEN 'Полная занятость' THEN 1
        WHEN 'Частичная занятость' THEN 2
        WHEN 'Стажировка' THEN 3
        WHEN 'Проектная работа' THEN 4
    END,
    CASE experience
        WHEN 'Нет опыта' THEN 1
        WHEN 'От 1 года до 3 лет' THEN 2
        WHEN 'От 3 до 6 лет' THEN 3
        WHEN 'Более 6 лет' THEN 4
    END;
'''

In [164]:
# результат запроса
df = pd.read_sql_query(query_additional_4, connection)
df

,employment,experience,vacancies_count,avg_salary
0,Полная занятость,Нет опыта,3803,45578
1,Полная занятость,От 1 года до 3 лет,12317,77505
2,Полная занятость,От 3 до 6 лет,5535,133730
3,Полная занятость,Более 6 лет,540,167297
4,Частичная занятость,Нет опыта,923,20920
5,Частичная занятость,От 1 года до 3 лет,351,45192
6,Частичная занятость,От 3 до 6 лет,80,89194
7,Частичная занятость,Более 6 лет,6,103859
8,Стажировка,Нет опыта,318,35764
9,Стажировка,От 1 года до 3 лет,36,50374


Рынок Data Science в РФ представлен узкой, но выскооплачиваемой нишей. Вакансии сосредоточены в крупных городах в европейской части РФ (Москва, СПб), где средние зарплаты составляют 208 и 173 тыс руб. Крупнейшие работодатели - Яндекс и реорганизованный Ростелеком, которые активно расширяют географию своего присутствия. Так, несмотря на концентрацию в столицах, есть предложения и в регионах. Примечательно, что другие компании топ-5 работодателей (Газпромнефть, Сбер и Тиньков) значительно меньше представлены в регионах

Есть сложности для людей без опыта: всего 51 вакансия. Низкая конкуренция на junior рынке компенсируется высоким спросом на middle специалистов, которым рынок в среднем готов платить 256 тыс руб/мес. В принципе, между градациями опыта "нет опыта", "от 1 до 3", "от 3 до 6" существует почти двукратное увеличение СЗП на каждом уровне, которое замедляется для overexperienced специалистов с опытом "более 6" лет до 25% (разница между 167 и 133 тыс для "полной занятости")

Также примечательно, что "проектная работа" для специалистов "от 3 до 6" и "нет опыта" оказывается выгоднее полноценной работы

Основным требованием для трудоустройства является Python

Итоги анализа как рекомендацию для соискателей можно подвести следующим абзацем:

Соискателям стоит строить карьерную траекторию с учетом географического фактора и постепенного перезода к более опытным позициям, где рост дохода становится значительным после трех лет профессиональной деятельности. При этом для максимизации дохода возможно рассмотрение проектной работы

In [86]:
connection.close()